<a href="https://colab.research.google.com/github/ganiosh92/DA-ecommerce-clickstream-analytics/blob/main/clickstream_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd
%cd /content/drive/My Drive/Sample-Data-From-Web/ecommerce_customer_behavior

/content
/content/drive/My Drive/Sample-Data-From-Web/ecommerce_customer_behavior


In [3]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize

In [4]:
customer = pd.read_csv('customer.csv')
clickstream = pd.read_csv('click_stream.csv')
transactions = pd.read_csv('transactions.csv')
product = pd.read_csv('product.csv', on_bad_lines='skip')

In [5]:
clickstream.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12833602 entries, 0 to 12833601
Data columns (total 6 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   session_id      object
 1   event_name      object
 2   event_time      object
 3   event_id        object
 4   traffic_source  object
 5   event_metadata  object
dtypes: object(6)
memory usage: 587.5+ MB


In [6]:
clickstream.head()

,session_id,event_name,event_time,event_id,traffic_source,event_metadata
0,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,HOMEPAGE,2019-09-06T15:54:32.821085Z,9c4388c4-c95b-4678-b5ca-e9cbc0734109,MOBILE,NaN
1,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,SCROLL,2019-09-06T16:03:57.821085Z,4690e1f5-3f99-42d3-84a5-22c4c4d8500a,MOBILE,NaN
2,7d440441-e67a-4d36-b324-80ffd636d166,HOMEPAGE,2019-09-01T12:05:10.322763Z,88aeaeb5-ec98-4859-852c-8abb483faf31,MOBILE,NaN
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri..."
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01T12:15:29.425431Z,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,{'payment_status': 'Success'}


In [7]:
#create a function to convert event_metadata column to JSON format
def func(a):
  return(json.loads(a.replace("'",'"')))

In [8]:
#flatten the event_metadata column

#drop rows with na values
clickstream_no_na = clickstream.dropna().copy()

#create a new column and fill it with dummy value = 1
clickstream_no_na['test_col'] = "1"

#apply the function to convert event_metadata column values from string to JSON
clickstream_no_na['test_col'] = clickstream_no_na['event_metadata'].apply(func)

#impute the flattened columns to the old dataset
clickstream_no_na_flattened = pd.concat([clickstream_no_na, json_normalize(clickstream_no_na['test_col'])], axis=1)

In [9]:
clickstream_no_na_flattened.head()

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,test_col,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri...","{'product_id': 15315, 'quantity': 4, 'item_pri...",6133.0,2.0,249443.0,NaN,NaN,NaN,NaN
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01T12:15:29.425431Z,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,{'payment_status': 'Success'},{'payment_status': 'Success'},6721.0,1.0,134504.0,NaN,NaN,NaN,NaN
5,7d440441-e67a-4d36-b324-80ffd636d166,SEARCH,2019-09-01T12:07:00.322763Z,c952142b-4fe9-4694-ad7f-21a5d1bed9ca,MOBILE,{'search_keywords': 'Dress Kondangan'},{'search_keywords': 'Dress Kondangan'},57622.0,1.0,384716.0,NaN,NaN,NaN,NaN
12,f7b5cd60-4f87-4b3c-8bde-c7da8930eec2,ADD_TO_CART,2019-09-09T05:06:03.425431Z,c1b42157-39a7-4521-a4dc-ed0200de301a,MOBILE,"{'product_id': 6133, 'quantity': 2, 'item_pric...","{'product_id': 6133, 'quantity': 2, 'item_pric...",NaN,NaN,NaN,Success,NaN,NaN,NaN
13,f7b5cd60-4f87-4b3c-8bde-c7da8930eec2,ADD_TO_CART,2019-09-09T05:23:36.425431Z,61cbe834-c585-406e-8eac-30f671e51fbb,MOBILE,"{'product_id': 6721, 'quantity': 1, 'item_pric...","{'product_id': 6721, 'quantity': 1, 'item_pric...",NaN,NaN,NaN,NaN,Tas Wanita,NaN,NaN


## TESTING

In [10]:
#Test
test = clickstream_no_na_flattened.copy()

In [11]:
#get the dataset where event_name is not a na/nan value
test2 = test[test["event_name"].isna() == False]

In [12]:
test2.head(1)

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,test_col,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri...","{'product_id': 15315, 'quantity': 4, 'item_pri...",6133.0,2.0,249443.0,NaN,NaN,NaN,NaN


In [13]:
#find distinct event_name types
set(test2['event_name'])

{'ADD_PROMO', 'ADD_TO_CART', 'BOOKING', 'SEARCH'}

In [34]:
#compute the number of "ADD_TO_CART" events that led to a buy
add_to_cart_num = len(test2[test2["event_name"] == 'ADD_TO_CART'])
add_to_cart_buy_num = len(test2[(test2['event_name'] == 'ADD_TO_CART') & (test2['payment_status'] == 'Success')])

#percentage to ADD_TO_CART's that resulted in a buy
atc_pct = round((add_to_cart_buy_num / add_to_cart_num) * 100, 2)

print(str(atc_pct) + "%")

5.06%


## Traffic Source

In [48]:
mobile_traffic = round((len(clickstream[clickstream['traffic_source'] == 'MOBILE']) / len(clickstream)) * 100, 2)
web_traffic = round((len(clickstream[clickstream['traffic_source'] == 'WEB']) / len(clickstream)) * 100, 2)

print("Percentage of Mobile Traffic = " + str(mobile_traffic)+"%")
print("Percentage of Web Traffic = " + str(web_traffic)+"%")

Percentage of Mobile Traffic = 89.98%
Percentage of Web Traffic = 10.02%


#Different Event Types and their Description

In [90]:
print(set(clickstream['event_name']))

#All Events
#1. HOMEPAGE = ------------- --> all N/A values
#2. CLICK = ------------- --> all N/A values
#3. ITEM_DETAIL = ------------- --> all N/A values
#4. SCROLL = ------------- --> all N/A values
#5. PROMO_PAGE = ------------- --> all N/A values
#6. BOOKING = payment, can result in either 'payment failed' or 'payment succeeded' --> no N/A values
#7. ADD_PROMO = Adding the Promo code and then seeing a promo amount BUT DID NOT RESULT IN A BUY--> no N/A values
#8. ADD_TO_CART = Act of adding certain products to cart BUT DID NOT RESULT IN A BUY. Has product_id, quantity and item_price --> no N/A values
#9. SEARCH = list of search keywords entered by the user --> No N/A values

print(" --- ")

print("% of Each Event")
print("1. Homepage: " + str(round(len(clickstream[clickstream['event_name'] == 'HOMEPAGE'])/len(clickstream),2)*100) + "%")
print("2. Click: " + str(round(len(clickstream[clickstream['event_name'] == 'CLICK'])/len(clickstream),2)*100)+ "%")
print("3. Item Detail: " + str(round(len(clickstream[clickstream['event_name'] == 'ITEM_DETAIL'])/len(clickstream),2)*100)+ "%")
print("4. Scroll: " + str(round(len(clickstream[clickstream['event_name'] == 'SCROLL'])/len(clickstream),2)*100)+ "%")
print("6. Booking: " + str(round(len(clickstream[clickstream['event_name'] == 'BOOKING'])/len(clickstream),2)*100)+ "%")
print("7. Add Promo: " + str(round(len(clickstream[clickstream['event_name'] == 'ADD_PROMO'])/len(clickstream),2)*100)+ "%")
print("8. Add to Cart: " + str(round(len(clickstream[clickstream['event_name'] == 'ADD_TO_CART'])/len(clickstream),2)*100)+ "%")
print("9. Search: " + str(round(len(clickstream[clickstream['event_name'] == 'SEARCH'])/len(clickstream),2)*100)+ "%")



{'SEARCH', 'BOOKING', 'ITEM_DETAIL', 'PROMO_PAGE', 'ADD_TO_CART', 'ADD_PROMO', 'CLICK', 'HOMEPAGE', 'SCROLL'}
 --- 
% of Each Event
1. Homepage: 19.0%
2. Click: 19.0%
3. Item Detail: 10.0%
4. Scroll: 13.0%
6. Booking: 7.000000000000001%
7. Add Promo: 3.0%
8. Add to Cart: 15.0%
9. Search: 9.0%


#Bounce Rate
the percentage of visitors who arrive at a digital storefront and “bounce,” leaving before ever progressing onto a second page. Bounce rates are calculated by dividing the number of one-page visits by the total number of visits to the site.

In [94]:
len(set(clickstream.session_id))

895203

In [98]:
len(clickstream[clickstream.session_id.isna() == False])

12833602

In [151]:
clickstream2 = pd.DataFrame(clickstream.groupby('session_id')['event_name'].agg(list))

In [152]:
clickstream2['event_name']

session_id
00001f29-3ede-4445-95f0-0178934e32ba    [HOMEPAGE, ADD_TO_CART, BOOKING, HOMEPAGE, CLI...
00003eca-954b-4150-aee1-63fc62f395cf         [HOMEPAGE, ADD_TO_CART, BOOKING, PROMO_PAGE]
0000425b-ddc8-4459-a240-6ba814dac132         [HOMEPAGE, ADD_TO_CART, BOOKING, PROMO_PAGE]
00005746-17ec-449c-9c5c-699a3ea9304a    [HOMEPAGE, ADD_TO_CART, BOOKING, CLICK, SCROLL...
0000649b-9514-4600-979b-42166a798996    [HOMEPAGE, ADD_TO_CART, ADD_PROMO, BOOKING, CL...
                                                              ...                        
ffffce53-1ba7-4d7d-a14a-ffa7079cfbff    [HOMEPAGE, ADD_TO_CART, ADD_TO_CART, ADD_PROMO...
ffffd3d9-79db-459e-a427-388204a7aac7    [HOMEPAGE, ADD_TO_CART, ADD_TO_CART, ADD_PROMO...
ffffdf8e-dea1-42bc-9839-7662675c0d38    [HOMEPAGE, ADD_TO_CART, BOOKING, HOMEPAGE, ITE...
fffff7cf-2a18-4303-9c12-aa836ae3b543    [HOMEPAGE, ADD_TO_CART, ADD_PROMO, BOOKING, CL...
fffffaf6-55ee-428b-a3bf-075cd7f1a093    [HOMEPAGE, ADD_TO_CART, ADD_PROMO, BOOKING, PR...

In [160]:
# Find rows where the 'event_name' column has just one key-value pair
filtered_df = clickstream2[clickstream2['event_name'].apply(lambda x: len(x) == 2)]

# Print the result
#print(filtered_df.sort_values(by = 'event_name', ascending = False))

                                                   event_name
session_id                                                   
82b8ced0-b70f-43a3-9c5b-128dfeec3a2e       [HOMEPAGE, SEARCH]
221229a0-b93c-41f1-9328-931daf61fb7f       [HOMEPAGE, SEARCH]
602ebeac-1e43-4a00-a59d-3b8c2fc0184c       [HOMEPAGE, SEARCH]
14390736-f21c-41b9-833b-f9f3fa89345a       [HOMEPAGE, SEARCH]
be3499ae-ab8b-47f9-8f99-8775278475c4       [HOMEPAGE, SEARCH]
...                                                       ...
aa3185a5-a2fa-41bf-825c-0248e6b61bc5  [HOMEPAGE, ADD_TO_CART]
aa242d29-d2ca-41e0-9887-f5386810721a  [HOMEPAGE, ADD_TO_CART]
1b3a15bd-f7fb-427a-a623-e7220fe63922  [HOMEPAGE, ADD_TO_CART]
a9a8ddaf-f059-4432-acd6-af02d9aa5362  [HOMEPAGE, ADD_TO_CART]
fff1f288-751d-442a-adfc-1afc7276c5c7  [HOMEPAGE, ADD_TO_CART]

[4794 rows x 1 columns]


['HOMEPAGE', 'SCROLL']

In [186]:
a = pd.DataFrame(filtered_df.reset_index())[['event_name']]
a

,event_name
0,"[HOMEPAGE, SCROLL]"
1,"[HOMEPAGE, SEARCH]"
2,"[HOMEPAGE, PROMO_PAGE]"
3,"[HOMEPAGE, CLICK]"
4,"[HOMEPAGE, ADD_TO_CART]"
...,...
4789,"[HOMEPAGE, SCROLL]"
4790,"[HOMEPAGE, CLICK]"
4791,"[HOMEPAGE, CLICK]"
4792,"[HOMEPAGE, ITEM_DETAIL]"


In [188]:
b = a[a['event_name'].apply(lambda events: events == ['HOMEPAGE', 'CLICK'])]
b

,event_name
3,"[HOMEPAGE, CLICK]"
10,"[HOMEPAGE, CLICK]"
11,"[HOMEPAGE, CLICK]"
15,"[HOMEPAGE, CLICK]"
19,"[HOMEPAGE, CLICK]"
...,...
4771,"[HOMEPAGE, CLICK]"
4774,"[HOMEPAGE, CLICK]"
4783,"[HOMEPAGE, CLICK]"
4790,"[HOMEPAGE, CLICK]"
